## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
import mlflow
import re
import matplotlib.pyplot as plt
import mlflow.pyfunc
import seaborn as sns
import tempfile
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import os


# Necessary configurations
spark = SparkSession.builder \
    .appName("Tweet Sentiment Analysis") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
dbutils = DBUtils(spark)

# Read the source file directory listing
files = dbutils.fs.ls(TWEET_SOURCE_PATH)

# Count the source files (how many are there?)
file_count = len(files)
print(f"Number of files in the source directory: {file_count}")

# Print the contents of one of the files
try:
    first_file_content = dbutils.fs.head(files[0].path)
    print(first_file_content)
except Exception as e:
    print(f"Error reading the file: {e}")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
# Define the schema for the raw data
raw_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True),
])

# Setup a read stream using cloudfiles and the source data format
raw_stream = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "json") \
    .schema(raw_schema) \
    .load(TWEET_SOURCE_PATH) \
    .withColumn("source_file", input_file_name()) \
    .withColumn("processing_time", current_timestamp())

#    .limit(100)
    
# Setup a write stream using cloudfiles to append to the bronze delta table
bronze_stream = raw_stream.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", BRONZE_CHECKPOINT) \
    .option("mergeSchema", "true") \
    .start(BRONZE_DELTA)


## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# How many tweets are captured in your Bronze Table?
total_tweets = bronze_df.count()
print(f"Total tweets in Bronze Table: {total_tweets}")

# Are there any columns that contain Nan or Null values? If so how many and what will you do in your silver transforms to address this?
null_counts = {col: bronze_df.filter(bronze_df[col].isNull()).count() for col in bronze_df.columns}
print(f"Null values per column: {null_counts}")

# Count the number of tweets by each unique user handle and sort the data by descending count.
tweets_per_user = bronze_df.groupBy("user").count()
sorted_tweets_per_user = tweets_per_user.orderBy(desc("count")).limit(20)

# How many tweets have at least one mention (@) how many tweet have no mentions (@)
tweets_with_mentions = bronze_df.filter("text like '%@%'").count()
tweets_without_mentions = bronze_df.filter("text not like '%@%'").count()
print(f"Tweets with mentions: {tweets_with_mentions}, Tweets without mentions: {tweets_without_mentions}")

# Show the top 20 tweeters
sorted_tweets_per_user.show()

top_tweeters = sorted_tweets_per_user.toPandas()

# Plot a bar chart that shows the top 20 tweeters (users)
plt.figure(figsize=(10, 6))
plt.bar(top_tweeters['user'], top_tweeters['count'], color='blue')
plt.xlabel('User Handle') 
plt.ylabel('Tweet Count')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show() 

In [0]:
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# Create a temp view
bronze_df.createOrReplaceTempView("bronze_view")

# Display the data field
display(spark.sql("""
        SELECT date, user, text, sentiment, source_file, processing_time
        FROM bronze_view
        LIMIT 100
        """))

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
# Setup a read stream on your bronze delta table
bronze_df = (
    spark.readStream
    .format("delta")
    .option("ignoreChanges", "true")
    .load(BRONZE_DELTA)
)

# Transform the bronze data to the silver data using the data definition at the top of the notebook
silver_stream = bronze_df \
    .withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss z yyyy")) \
    .withColumn("mentions", split(col("text"), " ")) \
    .withColumn("mention", explode("mentions")) \
    .withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", "")) \
    .filter(col("mention").rlike("^@\\w+")) \
    .select("timestamp", "mention", "cleaned_text", "sentiment") \
    .writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .start(SILVER_DELTA)


In [0]:
# # Check the stream's status
print(silver_stream.status)

In [0]:
silver_df = spark.read.format("delta").load(SILVER_DELTA)

silver_df.createOrReplaceTempView("silver_view")

# Display the data field
display(spark.sql("""
        SELECT timestamp, mention, cleaned_text, Sentiment
        FROM silver_view
        LIMIT 100
        """))

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
# Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
sentiment_analysis_udf = mlflow.pyfunc.spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")

# Setup a read stream on your silver delta table
silver_df = (
    spark.readStream
    .format("delta")
    .option("ignoreChanges", "true")
    .load(SILVER_DELTA)
)

# Use a spark UDF to parallelize the inference across your silver data
gold_df = silver_df.withColumn("predictions", sentiment_analysis_udf(col("cleaned_text"))) \
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment"),
        col("predictions.label").alias("predicted_sentiment"),
        col("predictions.score").alias("predicted_score"),
        when(col("sentiment") == "negative", 0).otherwise(1).alias("sentiment_id"),
        when(col("predictions.label") == "NEGATIVE", 0).otherwise(1)
        .alias("predicted_sentiment_id")
    )

# Setup a write stream to append to the gold delta table
gold_stream = gold_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .start(GOLD_DELTA)


In [0]:
print(gold_stream.status)

In [0]:
gold_table = spark.read.format("delta").load(GOLD_DELTA)
gold_table.createOrReplaceTempView("gold_view")

# Display the data field
display(spark.sql("""
        SELECT timestamp, mention, cleaned_text, sentiment, predicted_score, Predicted_sentiment, sentiment_id, predicted_sentiment_id
        FROM gold_view
        LIMIT 100
        """))

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
gold_table = spark.read.format("delta").load(GOLD_DELTA)

gold_filtered = gold_table.filter(col("predicted_sentiment") != "NEU").select(
    'sentiment_id', 'predicted_sentiment_id')

display(gold_filtered)

In [0]:
# Collect data
data = gold_filtered.toPandas()

# Calculate metrics
precision = precision_score(data['sentiment_id'], data['predicted_sentiment_id'], average='macro')
recall = recall_score(data['sentiment_id'], data['predicted_sentiment_id'], average='macro')
f1 = f1_score(data['sentiment_id'], data['predicted_sentiment_id'], average='macro')

In [0]:
# Generate confusion matrix
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(data['sentiment_id'], data['predicted_sentiment_id']), annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

In [0]:
# ENTER YOUR CODE HERE
# Save the confusion matrix to a temporary file
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
plt.savefig(temp_file.name)
plt.close()

with mlflow.start_run():
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log the confusion matrix as an artifact
    mlflow.log_artifact(temp_file.name, "confusion_matrix.png")

    # Log model details and Delta table version
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("model_version", "Model_Version")

    gold_table_version = DeltaTable.forPath(spark, GOLD_DELTA).history(1).select("version").first()[0]
    mlflow.log_param("delta_table_version", gold_table_version)

    silver_table_version = DeltaTable.forPath(spark, SILVER_DELTA).history(1).select("version").first()[0]
    mlflow.log_param("silver_delta_table_version", silver_table_version)


# Remove the temporary file
os.remove(temp_file.name)

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE
# Load Gold Delta table
gold_table = spark.read.format("delta").load(GOLD_DELTA)

# How many mentions are there in the gold data total?
num_mentions_count = gold_table.select("mention").distinct().count()

print(f"Total number of mentions: {num_mentions_count}")

In [0]:
# Count the number of negative, positive, and neutral tweets for each mention
num_sentiment_count = gold_table.groupBy("mention").agg(
    sum(when(col("Predicted_sentiment") == "NEG", 1).otherwise(0)).alias("negative_count"),
    sum(when(col("Predicted_sentiment") == "POS", 1).otherwise(0)).alias("positive_count"),
    sum(when(col("Predicted_sentiment") == "NEU", 1).otherwise(0)).alias("neutral_count")
)

# Add a column for total mentions
total_num_mentions = num_sentiment_count.withColumn("total_mentions", col("negative_count") + col("positive_count") + col("neutral_count"))

# Display the DataFrame
display(total_num_mentions)

In [0]:
sorted_num_mentions = total_num_mentions.orderBy(col("total_mentions"), ascending=False)

# Calculate the top 20 mentions with positive sentiment
top_20_positive_mentions = (
    sorted_num_mentions.orderBy(col("positive_count").desc()).limit(20)
)

# Calculate the top 20 mentions with negative sentiment
top_20_negative_mentions = (
    sorted_num_mentions.orderBy(col("negative_count").desc()).limit(20)
)

# Convert to Pandas for plotting
top_20_positive_pd = top_20_positive_mentions.toPandas()
top_20_negative_pd = top_20_negative_mentions.toPandas()

# Plotting positive mentions
plt.figure(figsize=(10, 5))
plt.bar(top_20_positive_pd['mention'], top_20_positive_pd['positive_count'], color='green')
plt.title('Top 20 Positive Mentions')
plt.xlabel('Mention')
plt.ylabel('Positive Count')
plt.xticks(rotation=45)
plt.show()

# Plotting negative mentions
plt.figure(figsize=(10, 5))
plt.bar(top_20_negative_pd['mention'], top_20_negative_pd['negative_count'], color='red')
plt.title('Top 20 Negative Mentions')
plt.xlabel('Mention')
plt.ylabel('Negative Count')
plt.xticks(rotation=45)
plt.show()


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE
stop_all_streams()

In [0]:
# Get the notebooks ending time
END_TIME = time.time()
Total_Time = END_TIME - START_TIME 

print(f"Total Time Taken: {Total_Time} seconds")

In [0]:
# from pyspark.sql import SparkSession
# import shutil

# # Spark Session
# spark = SparkSession.builder \
#     .appName("Delete Delta and Checkpoints") \
#     .getOrCreate()

# def delete_delta_and_checkpoints(delta_path, checkpoint_path):
#     # Delta Delta Table Path
#     try:
#         dbutils.fs.rm(delta_path, recurse=True)
#         print(f"Deleted delta table at {delta_path}")
#     except Exception as e:
#         print(f"Error deleting delta table at {delta_path}: {str(e)}")

#     # Delete Checkpoint Path
#     try:
#         dbutils.fs.rm(checkpoint_path, recurse=True)
#         print(f"Deleted checkpoint at {checkpoint_path}")
#     except Exception as e:
#         print(f"Error deleting checkpoint at {checkpoint_path}: {str(e)}")

# delta_table_path = 
# checkpoint_path = 

# delete_delta_and_checkpoints(delta_table_path, checkpoint_path)

# BRONZE_DELTA BRONZE_CHECKPOINT
# SILVER_DELTA SILVER_CHECKPOINT
# GOLD_DELTA GOLD_CHECKPOINT


## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE